In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import math

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split


2023-11-17 08:54:52.984220: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-17 08:54:53.021830: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-17 08:54:53.021864: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-17 08:54:53.022577: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-17 08:54:53.027724: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-17 08:54:53.029110: I tensorflow/core/platform/cpu_feature_guard.cc:1

# Load Data

First step is to ingest all the data we have available and merge them into a flattened datastructure containing all measurements. Indexes are ignored and rewritten to allow all readings to be added to the DF:

In [2]:
# parse the labels.csv
labels = pd.read_csv('labels.csv', index_col=0)
labels = labels.sort_values('id')

# grab filenames from the data directory
filenames = os.listdir('data')
filenames.sort()

dataframes = []

# parse and concatenate all csv files into df
for filename in filenames:
  if filename.endswith('.csv'):
    batch = pd.read_csv(os.path.join('data',filename), index_col=0)
    batch['batch'] = int(filename.replace('.csv', ''))
    dataframes.append(batch)

df = pd.concat(dataframes, ignore_index=True)

# clean up original dataframes
del dataframes

# add label column (if it is not already available)
if (not 'label' in df.columns):
  df = df.merge(labels, left_on=["batch"], right_on=["id"])


In [3]:
def time_to_float(inputstr):
  hours, minutes, seconds = map(float, inputstr.split(':'))

  # return hours * 3600 + minutes * 60 + seconds
  # this is sufficient because hours should always be 0
  return minutes * 60 + seconds

if (not df['sensorid'].dtype == 'int'):
  df['sensorid'] = df['sensorid'].astype('int')
if (not df['label'].dtype == 'category'):
  df['label'] = df['label'].astype('category')
if (not df['zeit'].dtype == 'float64'):
  df['zeit'] = df['zeit'].apply(time_to_float)

# print(df[:10])
# print(labels[:10])


# Test Data Preprocessing

In [4]:
SEQUENCE_LENGTH = 128

sequences = []
sequence_labels = []

for batch, readings in df.groupby('batch'):
  readings = readings.sort_values('zeit')
  for i in range(0, len(readings) - SEQUENCE_LENGTH, SEQUENCE_LENGTH):
    sequence = readings.iloc[i:i + SEQUENCE_LENGTH]
    sequences.append(sequence[['zeit', 'sensorid', 'messwert']].values)
    sequence_labels.append(sequence['label'].values[0])

sequences = np.array(sequences)
sequence_labels = np.array(sequence_labels)

X_train, X_test, y_train, y_test = train_test_split(sequences, sequence_labels, test_size=0.2, random_state=42)
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


# Modelling

In [5]:
# set a bunch of constants that help configure the model and training
BATCH_SIZE = 64
LSTM_UNITS = 128

CHECKPOINT_PATH = '.checkpoints/cp-{epoch:03d}.ckpt'
# CHECKPOINT_DIR = os.path.dirname(CHECKPOINT_PATH)

N_BATCHES = math.ceil(len(X_train) / BATCH_SIZE)
num_features = X_train.shape[2]
num_classes = len(labels['label'].unique())

model = Sequential()
model.add(LSTM(LSTM_UNITS, input_shape=(SEQUENCE_LENGTH, num_features)))
model.add(Dense(num_classes, activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Training
cp_cb = ModelCheckpoint(filepath=CHECKPOINT_PATH, save_weights_only=True, save_freq=8*N_BATCHES)
stp_cb = EarlyStopping(monitor='loss', patience=8, restore_best_weights=True, min_delta=1e-4, start_from_epoch=32, verbose=1)

# model.fit(X_train, y_train, epochs=256, batch_size=BATCH_SIZE, callbacks=[cp_cb, stp_cb], validation_data=(X_val, y_val))
model.fit(X_train, y_train, epochs=256, batch_size=BATCH_SIZE, callbacks=[cp_cb, stp_cb])

# Save Model
model.save(f'models/lstm_{LSTM_UNITS}-seq_{SEQUENCE_LENGTH}-batch_{BATCH_SIZE}.keras')



2023-11-17 08:55:16.332338: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-17 08:55:16.332886: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/256


2023-11-17 08:55:16.919938: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 30236160 exceeds 10% of free system memory.


308/308 [==============================] - 32s 100ms/step - loss: 1.1045 - accuracy: 0.3425 - val_loss: 1.1004 - val_accuracy: 0.3362
Epoch 2/256
308/308 [==============================] - 30s 99ms/step - loss: 1.0975 - accuracy: 0.3581 - val_loss: 1.0940 - val_accuracy: 0.3673
Epoch 3/256
308/308 [==============================] - 30s 97ms/step - loss: 1.0950 - accuracy: 0.3643 - val_loss: 1.0994 - val_accuracy: 0.3560
Epoch 4/256
308/308 [==============================] - 30s 96ms/step - loss: 1.0943 - accuracy: 0.3638 - val_loss: 1.0975 - val_accuracy: 0.3539
Epoch 5/256
308/308 [==============================] - 30s 96ms/step - loss: 1.0940 - accuracy: 0.3654 - val_loss: 1.0932 - val_accuracy: 0.3580
Epoch 6/256
308/308 [==============================] - 30s 96ms/step - loss: 1.0916 - accuracy: 0.3697 - val_loss: 1.0975 - val_accuracy: 0.3665
Epoch 7/256
308/308 [==============================] - 30s 99ms/step - loss: 1.0913 - accuracy: 0.3702 - val_loss: 1.0970 - val_accuracy: 0.3

# Evaluation

Best current model `lstm_100-seq_128-batch_64` (74.74%)

In [6]:
print(model.predict(X_test[:5]))
loss, acc = model.evaluate(X_test, y_test, verbose=2)

print("Model accuracy: {:5.2f}%".format(100 * acc))


1/1 [==============================] - 0s 410ms/step
[[0.4175435  0.44573614 0.1367203 ]
 [0.70361423 0.18829243 0.10809335]
 [0.34088692 0.6448194  0.0142938 ]
 [0.39665085 0.5079025  0.09544659]
 [0.4197015  0.46429542 0.11600314]]
193/193 - 5s - loss: 0.6285 - accuracy: 0.6848 - 5s/epoch - 27ms/step
Model accuracy: 68.48%


In [7]:
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               67584     
                                                                 
 dense (Dense)               (None, 3)                 387       
                                                                 
Total params: 67971 (265.51 KB)
Trainable params: 67971 (265.51 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
